In [1]:
%matplotlib inline
import scipy.stats as stats                    # used for normal dist
from IPython.core.pylabtools import figsize
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc as pm                               # used for creating MC / Bayes estimate 
import scipy.optimize as sop                    # used for finding minimum - optimization function
import os
os.chdir('/users/tofaj/Downloads')
os.getcwd()

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


'C:\\users\\tofaj\\Downloads'

In [2]:
fashion_train = pd.read_csv('fashion-mnist_train.csv') #load in training dataset
fashion_train = pd.DataFrame(fashion_train)
fashion_test = pd.read_csv('fashion-mnist_test.csv') #load in test dataset
fashion_test = pd.DataFrame(fashion_test)
fashion_train.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
fashion_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
null_values = fashion_train.isnull().sum #check for null values
null_values

<bound method DataFrame.sum of        label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      False   False   False   False   False   False   False   False   False   
1      False   False   False   False   False   False   False   False   False   
2      False   False   False   False   False   False   False   False   False   
3      False   False   False   False   False   False   False   False   False   
4      False   False   False   False   False   False   False   False   False   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
59995  False   False   False   False   False   False   False   False   False   
59996  False   False   False   False   False   False   False   False   False   
59997  False   False   False   False   False   False   False   False   False   
59998  False   False   False   False   False   False   False   False   False   
59999  False   False   False   False   False   False   False   False   False   

       p

In [5]:
fashion_train.describe() #descriptive statistics

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.00000
mean,4.500000,0.000900,0.006150,0.035333,0.101933,0.247967,0.411467,0.805767,2.198283,5.682000,...,34.625400,23.300683,16.588267,17.869433,22.814817,17.911483,8.520633,2.753300,0.855517,0.07025
std,2.872305,0.094689,0.271011,1.222324,2.452871,4.306912,5.836188,8.215169,14.093378,23.819481,...,57.545242,48.854427,41.979611,43.966032,51.830477,45.149388,29.614859,17.397652,9.356960,2.12587
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,4.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,58.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,9.000000,16.000000,36.000000,226.000000,164.000000,227.000000,230.000000,224.000000,255.000000,254.000000,...,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,170.00000


In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import accuracy_score

X_train = fashion_train.drop('label', axis=1) #Target column for prediction is Pixel775
y_train = fashion_train['label']

X_test = fashion_test.drop('label', axis=1)
y_test = fashion_test['label']

model = GaussianNB() #training the model
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

modelacc = metrics.accuracy_score(y_test, y_pred)
print("The model accuracy is:", modelacc)

C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

The model accuracy is: 0.5914


C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


For naive bayes, the accuracy is low at 0.5914 (59 percent). This indicates naive bayes might not be the best for predictions since the accuracy is just under 60 percent which is seen as relatively low. It could be due to a variety of reasons but the model most likely suffers from overfitting since the complexity is limited. Despite the low performance, the model is faster than most in terms of processing speed.

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier() #training model using decision tree classifier
tree_model.fit(X_train, y_train)

Y_pred_tree = tree_model.predict(X_test)

tree_acc = metrics.accuracy_score(y_test, Y_pred_tree)
print('Accuracy:',tree_acc)

C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

Accuracy: 0.7955


C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\tofaj\Downloads\New folder\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

The accuracy for the decision tree model is about 80%. Decision trees capture more complex relationships compared to naive bayes which thrives on simplicity and feature independence. Decision trees also handle features that aren't as relevant much better than naive bayes because they can automatically ignore the features that are irrelevant. Overall it's better suited for this particular dataset however they still suffer from limitations such as overfitting and noise. The model isn't as fast as naive bayes but it is not too slow.

In [9]:
import tensorflow as tf
## Neural Network

#normalizing the pixel values to range
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

#reshaping the data
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

#model architecture, 2 convolutional layers, 2 max pooling and 2 dense layers for output
model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)), 
                             tf.keras.layers.MaxPooling2D((2,2)),
                             tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2,2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(10, activation='softmax')])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

#performance assessment
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', test_accuracy)

Epoch 1/5
844/844 [==============================] - 20s 23ms/step - loss: 1.0176 - accuracy: 0.6309 - val_loss: 0.7335 - val_accuracy: 0.7332
Epoch 2/5
844/844 [==============================] - 19s 22ms/step - loss: 0.6661 - accuracy: 0.7475 - val_loss: 0.6544 - val_accuracy: 0.7550
Epoch 3/5
844/844 [==============================] - 18s 22ms/step - loss: 0.5939 - accuracy: 0.7772 - val_loss: 0.6049 - val_accuracy: 0.7800
Epoch 4/5
844/844 [==============================] - 18s 22ms/step - loss: 0.5390 - accuracy: 0.7971 - val_loss: 0.5468 - val_accuracy: 0.7960
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.4779 - accuracy: 0.8260
Accuracy: 0.8259999752044678


The neural network offers more advantages than both of the previous models. They are mostly known for their ability to capture complex patterns and are suitable for large datasets with complex relationships. This model's accuracy is about 83% and the more convolutional layers, the higher. It is more time consuming than the previous two but gaining higher accuracy is worth it. There is still the fear of the model suffering from overfitting which would reduce it's accuracy but it is remarkably successful in feature learning.